In [39]:
# Install libraries the first time
# ! pip install -q yfinance pandas

In [40]:
import yfinance as yf
import pandas as pd

### Variables

In [41]:
symbols = [
    'AAPL',  # Apple Inc.
    'MSFT',  # Microsoft Corporation
    'AMZN',  # Amazon.com, Inc.
    'GOOGL', # Alphabet Inc. (Class A)
    'META',  # Meta Platforms, Inc.
    'TSLA',  # Tesla, Inc.
    'NVDA',  # NVIDIA Corporation
    'NVO'
]

timeFrame = '2y'  # Options: 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max

### Getting historic data

In [42]:
# Define a function to calculate FutureYearChange
def calculate_future_year_change(ticker_symbol, timeframe, buffer=1):
    """
    Calculate Future Year Change for a ticker, requesting a larger valid period to ensure full data coverage.
    """
    # Map the valid period for yfinance
    valid_periods = ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
    extended_timeframe = 'max' if timeframe == 'max' else valid_periods[valid_periods.index(timeframe) + buffer]

    # Get historical data
    ticker = yf.Ticker(ticker_symbol)
    futureChange = ticker.history(period=extended_timeframe)
    
    # Calculate Future Year Change
    futureChange['Future Year Change'] = (futureChange['Close'].shift(-252) / futureChange['Close'] - 1)

    # Define the valid date range: 2 years ago to 1 year ago
    end_date = futureChange.index[-1] - pd.DateOffset(years=1)  # 1 year ago
    start_date = end_date - pd.DateOffset(years=int(timeframe[:-1]))  # 2 years ago
    futureChange = futureChange.loc[start_date:end_date]

    # Drop rows where Future Year Change is NaN
    futureChange = futureChange.dropna(subset=['Future Year Change'])

    return futureChange

# Data collection
histData = pd.DataFrame()
for symbol in symbols:
    try:
        futureChange = calculate_future_year_change(symbol, timeFrame)
        futureChange['Ticker'] = symbol  # Add Ticker column
        histData = pd.concat([histData, futureChange])
    except Exception as e:
        print(f"Error processing {symbol}: {e}")

# Reorder columns to have 'Ticker' right after the date
histData.reset_index(inplace=True)  # Move the date index to a column
columns_order = ['Date', 'Ticker'] + [col for col in histData.columns if col not in ['Date', 'Ticker']]
histData = histData[columns_order]

# Save to CSV
histData.to_csv('data.csv', index=False)

In [43]:
def get_static_ev_data(ticker_symbol):
    """
    Fetch static data needed for EV calculation: Total Debt, Cash, and Shares Outstanding.
    """
    try:
        ticker = yf.Ticker(ticker_symbol)
        info = ticker.info

        # Retrieve static components
        total_debt = info.get('totalDebt', 0)  # Defaults to 0 if not available
        cash = info.get('totalCash', 0)       # Defaults to 0 if not available
        shares_outstanding = info.get('sharesOutstanding', None)  # None if not available

        print(f"Static Data for {ticker_symbol}:")
        print(f"  Total Debt: {total_debt}")
        print(f"  Cash: {cash}")
        print(f"  Shares Outstanding: {shares_outstanding}")

        return total_debt, cash, shares_outstanding
    except Exception as e:
        print(f"Error fetching EV components for {ticker_symbol}: {e}")
        return None, None, None


In [44]:
def calculate_approx_ebit(ticker_symbol):
    """
    Approximate EBIT using Revenue and Operating Income (if available).
    """
    try:
        ticker = yf.Ticker(ticker_symbol)
        info = ticker.info

        # Try fetching revenue and operating income
        revenue = info.get('totalRevenue', None)
        operating_income = info.get('operatingIncome', None)

        if revenue and operating_income:
            return operating_income  # Use direct value if available
        elif revenue:
            # If operating income is unavailable, assume a fixed margin
            estimated_margin = 0.15  # Example: 15% operating margin
            return revenue * estimated_margin
        else:
            return None
    except Exception as e:
        print(f"Error calculating EBIT for {ticker_symbol}: {e}")
        return None


In [45]:
def fetch_ebit(ticker_symbol):
    """
    Fetch or approximate EBIT for the ticker symbol.
    """
    ebit = calculate_approx_ebit(ticker_symbol)
    if ebit is not None:
        print(f"Approximated EBIT for {ticker_symbol}: {ebit}")
    else:
        print(f"EBIT for {ticker_symbol} is unavailable.")
    return ebit


In [46]:
def calculate_daily_ev_ebit_for_histdata(hist_data):
    """
    Add a column for daily EV/EBIT to existing historical data.
    """
    data_with_ev_ebit = hist_data.copy()
    data_with_ev_ebit['EV/EBIT'] = None  # Initialize the column

    for symbol in data_with_ev_ebit['Ticker'].unique():
        # Filter data for the current ticker
        ticker_data = data_with_ev_ebit[data_with_ev_ebit['Ticker'] == symbol].copy()

        # Fetch static EV data
        total_debt, cash, shares_outstanding = get_static_ev_data(symbol)

        if shares_outstanding is None:
            print(f"Missing shares outstanding for {symbol}. Skipping EV/EBIT calculation.")
            continue

        # Fetch EBIT
        ebit = fetch_ebit(symbol)

        if ebit is None or ebit == 0:
            print(f"Missing or invalid EBIT for {symbol}. Skipping EV/EBIT calculation.")
            continue

        # Calculate EV and EV/EBIT
        ticker_data['EV'] = (ticker_data['Close'] * shares_outstanding) + total_debt - cash
        ticker_data['EV/EBIT'] = ticker_data['EV'] / ebit

        # Update the main DataFrame
        data_with_ev_ebit.loc[ticker_data.index, 'EV/EBIT'] = ticker_data['EV/EBIT']

    # Drop intermediate columns if present
    if 'EV' in data_with_ev_ebit.columns:
        data_with_ev_ebit.drop(columns=['EV'], inplace=True)

    return data_with_ev_ebit


In [48]:
newhistData = calculate_daily_ev_ebit_for_histdata(histData)

# Save updated data to a CSV file
newhistData.to_csv('data.csv', index=False)

newhistData.head()

Static Data for AAPL:
  Total Debt: 119058997248
  Cash: 65171001344
  Shares Outstanding: 15037899776
Approximated EBIT for AAPL: 58655249203.2
Static Data for MSFT:
  Total Debt: 96838000640
  Cash: 78429003776
  Shares Outstanding: 7434880000
Approximated EBIT for MSFT: 38128499097.6
Static Data for AMZN:
  Total Debt: 158534991872
  Cash: 88050999296
  Shares Outstanding: 10515000320
Approximated EBIT for AMZN: 93019196620.8
Static Data for GOOGL:
  Total Debt: 29289000960
  Cash: 93229998080
  Shares Outstanding: 5842999808
Approximated EBIT for GOOGL: 50978852044.799995
Static Data for META:
  Total Debt: 49046999040
  Cash: 70899998720
  Shares Outstanding: 2180000000
Approximated EBIT for META: 23434049126.399998
Static Data for TSLA:
  Total Debt: 12782999552
  Cash: 33648001024
  Shares Outstanding: 3210060032
Approximated EBIT for TSLA: 14572500172.8
Static Data for NVDA:
  Total Debt: 10014999552
  Cash: 34800001024
  Shares Outstanding: 24490000384
Approximated EBIT for NV

,Date,Ticker,Open,High,Low,Close,Volume,Dividends,Stock Splits,Future Year Change,EV/EBIT
0,2022-01-18 00:00:00-05:00,AAPL,168.745817,169.759216,166.679671,167.063385,90956700,0.0,0.0,-0.198733,43.750056
1,2022-01-19 00:00:00-05:00,AAPL,167.260194,168.322790,163.265629,163.550949,94815000,0.0,0.0,-0.165794,42.849546
2,2022-01-20 00:00:00-05:00,AAPL,164.288820,166.945302,161.533944,161.858627,91420500,0.0,0.0,-0.137262,42.415672
3,2022-01-21 00:00:00-05:00,AAPL,161.770049,163.649270,159.684222,159.792450,122848900,0.0,0.0,-0.117313,41.88595
4,2022-01-24 00:00:00-05:00,AAPL,157.441024,159.684277,152.206757,159.015228,162294600,0.0,0.0,-0.117168,41.686688
